This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [ ]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

Loading of the necessary modules

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from flip import fitter, plot_utils, utils, vectors
from flip.covariance import covariance, contraction
from pkg_resources import resource_filename
flip_base = resource_filename("flip", ".")
data_path = os.path.join(flip_base, "data")
plt.style.use(os.path.join(data_path,"style.mplstyle"))

Loading the data, located in the package itself



In [ ]:
ktt, ptt = np.loadtxt(os.path.join(data_path, "power_spectrum_tt.txt"))
kmt, pmt = np.loadtxt(os.path.join(data_path, "power_spectrum_mt.txt"))
kmm, pmm = np.loadtxt(os.path.join(data_path, "power_spectrum_mm.txt"))

# the extra Koda et al. damping term Eq 33 from https://arxiv.org/pdf/2303.01198
sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}


First part of the flip package: fast computation of the covariance matrix based on theory and SNIa coordinates

In [ ]:
filename = "abacussample_s500_r300_b20_d0.05_i0.1_xyz000_lnone_go.hdf5"
import h5py
import numpy
from astropy.cosmology import Planck18 as cosmo

def read_abacus(filename):
    with h5py.File(filename, "r") as f:
        data=f['abacussample']
        zlist = list(data['z']) # observed redshift
        ra = list(data['ra'])
        dec = list(data['dec'])
        rcom_zobs = cosmo.comoving_distance(numpy.array(zlist)).value
        # cosmoz = list(data['cosmoz'])
        
    return numpy.array([ra,dec,rcom_zobs])

In [ ]:
coordinates_velocity = read_abacus(filename)

In [ ]:
size_batch=10_000
number_worker=16


covariance_fit = covariance.CovMatrix.init_from_flip(
    "carreres23",
    "velocity",
    power_spectrum_dict,
    coordinates_velocity=coordinates_velocity,
    size_batch=size_batch,
    number_worker=number_worker,
)


In [ ]:
covariance_fit.write("temp","pickle")